In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Load Data
vehicles_df = pd.read_csv('vehicles_df.csv')

# Load Reviews Data (Kia Motors specific)
reviews_df = pd.read_csv('car_reviews_with_sentiment_and_percentage.csv')

# Initialize Dash App
app = dash.Dash(__name__)

# Geocode cities to get latitude and longitude
geolocator = Nominatim(user_agent="geoapi")
city_coordinates = []

for city in city_counts['City'].unique():
    try:
        location = geolocator.geocode(f"{city}, Illinois")  # Update state/region as needed
        if location:
            city_coordinates.append({
                "City": city,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
        else:
            city_coordinates.append({
                "City": city,
                "latitude": None,
                "longitude": None
            })
    except GeocoderTimedOut:
        city_coordinates.append({
            "City": city,
            "latitude": None,
            "longitude": None
        })

# Layout
app.layout = html.Div([
    # Dashboard Header
    html.Div([
        html.H1("Vehicle and Review Insights Dashboard", style={
            'textAlign': 'center', 
            'color': '#F1F1F1',
            'fontSize': '36px'
        }),
        html.P("Explore vehicle registrations and customer reviews interactively.", style={
            'textAlign': 'center', 
            'color': '#A5D8DD',
            'fontSize': '18px'
        })
    ], style={
        'backgroundColor': '#202020',
        'padding': '20px',
        'borderBottom': '5px solid #1C4E80'
    }),
     # Introduction Section
    html.Div([
        html.H2("Introduction: Public Passenger Vehicles in Illinois", style={'color': '#1C4E80'}),
        html.P(
            "The transportation industry in Illinois includes a wide array of public passenger vehicles, "
            "each catering to the diverse commuting and travel needs of residents and visitors. These vehicles, "
            "ranging from taxicabs and liveries to ambulances, medicars, charter and sightseeing buses, "
            "horse-drawn carriages, and pedicabs, are operated by licensed chauffeurs to transport passengers for hire.",
            style={'color': '#7E909A'}
        ),
        html.P(
            "To improve the understanding and management of these vital services, comprehensive data on registered "
            "public passenger vehicles is systematically collected. This data offers detailed insights into various "
            "vehicle attributes and operational aspects, enabling stakeholders to analyze trends, ensure compliance, "
            "and optimize services. This study focuses on exploring this rich dataset to derive actionable insights "
            "into the public transportation landscape in Illinois.",
            style={'color': '#7E909A'}
        ),
        html.H2("Introduction: Kia Customer Reviews", style={'color': '#1C4E80'}),
        html.P(
            "Customer feedback plays a crucial role in understanding brand perception and improving vehicle quality "
            "and services. To gain insights into customer experiences, a dataset containing 7,000 reviews of Kia vehicles "
            "listed on Edmunds automotive inventory has been provided. This dataset offers a comprehensive collection of "
            "customer opinions, touching on aspects such as driving experience, features, value for money, and potential issues.",
            style={'color': '#7E909A'}
        ),
        html.P(
            "By analyzing these reviews, we aim to uncover patterns in customer satisfaction, identify frequently discussed "
            "topics, and assess the overall sentiment toward Kia vehicles. This analysis will involve review classification "
            "and sentiment analysis, providing valuable insights into customer preferences and areas for improvement. Such "
            "findings can guide Kia Motors in enhancing their offerings and strengthening customer relationships.",
            style={'color': '#7E909A'}
        )
    ], style={'backgroundColor': '#F1F1F1', 'padding': '20px', 'marginBottom': '20px'}),

    # Dropdowns
    html.Div([
        html.Div([
            html.Label("Filter by Fuel Type:", style={
                'fontSize': '16px', 
                'fontWeight': 'bold', 
                'color': '#202020'
            }),
            dcc.Dropdown(
                id='fuel-filter',
                options=[{'label': ft, 'value': ft} for ft in vehicles_df['Vehicle Fuel Source'].dropna().unique()],
                value=vehicles_df['Vehicle Fuel Source'].dropna().unique()[0],
                clearable=False,
                style={'backgroundColor': '#F1F1F1', 'border': '1px solid #7E909A'}
            )
        ], style={'padding': '20px', 'width': '48%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Filter by Sentiment:", style={
                'fontSize': '16px', 
                'fontWeight': 'bold', 
                'color': '#202020'
            }),
            dcc.Dropdown(
                id='sentiment-dropdown',
                options=[
                    {'label': 'All Sentiments', 'value': 'all'},
                    {'label': 'Positive', 'value': 'POSITIVE'},
                    {'label': 'Negative', 'value': 'NEGATIVE'}
                ],
                value='all',
                clearable=False,
                style={'backgroundColor': '#F1F1F1', 'border': '1px solid #7E909A'}
            )
        ], style={'padding': '20px', 'width': '48%', 'display': 'inline-block'})
    ], style={'backgroundColor': '#F1F1F1'}),

    # Graphs
    html.Div([
        html.Div([
            dcc.Graph(id='sunburst-chart'),
            # html.P(
            #     "This chart displays the hierarchy of vehicles by make and model. The size of each segment "
            #     "represents the count of vehicles within that category, helping to visualize the distribution.",
            #     style={'textAlign': 'center', 'color': '#7E909A'}
            # ),
            html.P(
                "Electric vehicles are primarily manufactured by Tesla, Mainstreet, Tuk Tuk, and Polaris in Illinois state."               ,
                style={'textAlign': 'center', 'color': '#7E909A'}
            ),


        ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Graph(id='city-map'),
            html.P(
                "This map shows the geographical distribution of vehicle registrations across different cities. "
                "The size of the marker represents the count of vehicles registered in that city.",
                style={'textAlign': 'center', 'color': '#7E909A'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
    ], style={'backgroundColor': '#F1F1F1', 'padding': '20px'}),

    html.Div([
        html.Div([
            dcc.Graph(id='histogram-chart'),
            html.P(
                " From the registered public passenger vehicles in Illinois state, only 5 electric vehicles are manufactured before 2010 and other vehicles are manufactured after 2015",
                style={'textAlign': 'center', 'color': '#7E909A'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Graph(id='classification-pie-chart'),
            # html.P(
            #     "This pie chart categorizes the topics discussed in Kia vehicle reviews. Each slice represents "
            #     "the proportion of reviews discussing a specific topic.",
            #     style={'textAlign': 'center', 'color': '#7E909A'}
            # ),
            html.P(
                "According to the customer reviews, more reviews talk about the issues, and the percentage is 31.%. Talks about the other of the KIA car model are the lowest number of reviews appeared in the reviews dataset.",
                style={'textAlign': 'center', 'color': '#7E909A'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
    ], style={'backgroundColor': '#F1F1F1', 'padding': '20px'}),

    html.Div([
        html.Div([
            dcc.Graph(id='boxplot-chart'),
            html.P(
                "This box plot shows the spread of vehicle model years for each fuel type. It highlights the "
                "range, median, and potential outliers in the data.",
                style={'textAlign': 'center', 'color': '#7E909A'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Graph(id='line-chart'),
            # html.P(
            #     "This line chart illustrates the average sentiment percentage for each review topic in Kia vehicle reviews, "
            #     "indicating the overall customer satisfaction trend.",
            #     style={'textAlign': 'center', 'color': '#7E909A'}
            # ),
            html.P(
                "The average sentiment percentage for the 'driving experience' topic is 94%, which is the highest among all the topics. The 'Other' topic has the lowest average sentiment percentage of 94%",
                style={'textAlign': 'center', 'color': '#7E909A'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
    ], style={'backgroundColor': '#F1F1F1', 'padding': '20px'}),

    html.Div([
        dcc.Graph(id='sentiment-bar-chart'),
        html.P(
            "This bar chart visualizes the sentiment analysis of Kia vehicle reviews, showing the count of positive "
            "and negative reviews in the dataset.",
            style={'textAlign': 'center', 'color': '#7E909A'}
        ),
        html.P(
            "After sentiment analysis, a higher number of customer reviews for KIA vehicles listed on Edmunds Automotive inventory are positive. All the reviews were categorized as positive and negative. Therefore, the 'error' review is zero.",
            style={'textAlign': 'center', 'color': '#7E909A'}
        ),


    ], style={'padding': '20px', 'backgroundColor': '#F1F1F1'}),

    # Footer
    html.Div([
        html.P("© 2024 Vehicle Insights Dashboard. All rights reserved.", style={
            'textAlign': 'center',
            'color': '#A5D8DD',
            'fontSize': '14px'
        })
    ], style={
        'backgroundColor': '#202020', 
        'padding': '10px'
    })
])

# Callbacks
@app.callback(
    [Output('sunburst-chart', 'figure'),
     Output('city-map', 'figure'),
     Output('classification-pie-chart', 'figure'),
     Output('sentiment-bar-chart', 'figure'),
     Output('histogram-chart', 'figure'),
     Output('boxplot-chart', 'figure'),
     Output('line-chart', 'figure')],
    [Input('fuel-filter', 'value'),
     Input('sentiment-dropdown', 'value')]
)
def update_charts(fuel_type, selected_sentiment):
    # Filter data
    filtered_data = vehicles_df[vehicles_df['Vehicle Fuel Source'] == fuel_type].copy()
    filtered_data['Count'] = 1

    # Handle missing values
    filtered_data['Vehicle Make'] = filtered_data['Vehicle Make'].fillna('Unknown')
    filtered_data['Vehicle Model'] = filtered_data['Vehicle Model'].fillna('Unknown')

    # Sunburst Chart
    sunburst = px.sunburst(
        filtered_data, 
        path=['Vehicle Make', 'Vehicle Model'], 
        values='Count', 
        color_discrete_sequence=px.colors.qualitative.Pastel
    )

    # City Map
    city_map = px.scatter_mapbox(
        city_data, 
        lat="latitude", 
        lon="longitude", 
        size="Count", 
        color="Count", 
        mapbox_style="carto-positron", 
        zoom=6
    )

    # Histogram
    histogram = px.histogram(
        filtered_data, 
        x='Vehicle Model Year', 
        nbins=20, 
        title="Vehicle Model Year Distribution", 
        labels={'Vehicle Model Year': 'Year'}, 
        color_discrete_sequence=['#A5D8DD']
    )

    # Classification Pie
    filtered_reviews = reviews_df if selected_sentiment == 'all' else reviews_df[reviews_df['sentiment'] == selected_sentiment]
    classification_pie = px.pie(filtered_reviews, names='talks_about', title="Review Topics")

    # Sentiment Bar
    sentiment_bar = px.bar(
        filtered_reviews.groupby('sentiment').size().reset_index(name='Count'), 
        x='sentiment', y='Count', color='sentiment', 
        title="Sentiment Analysis", 
        color_discrete_sequence=['#EA6A47', '#1C4E80']
    )

    # Box Plot
    boxplot = px.box(
        filtered_data, 
        x='Vehicle Fuel Source', 
        y='Vehicle Model Year', 
        title="Vehicle Model Year by Fuel Type", 
        color='Vehicle Fuel Source', 
        color_discrete_sequence=px.colors.qualitative.Set3
    )

    # Line Chart
    line_chart = px.line(
        reviews_df.groupby('talks_about').agg({'sentiment_percentage': 'mean'}).reset_index(),
        x='talks_about', y='sentiment_percentage',
        title="Average Sentiment Percentage by Review Topic",
        color_discrete_sequence=['#0091D5']
    )

    return sunburst, city_map, classification_pie, sentiment_bar, histogram, boxplot, line_chart

# Run the App
if __name__ == '__main__':
    webbrowser.open_new('http://127.0.0.1:8090/')
    app.run_server(debug=True, host='127.0.0.1', port=8090)
